# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`).

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [ ]:
import string

data = pd.read_csv('preprocessed_descriptions.csv')
description = data.preprocessed_descriptions.astype(str)

words = []
for i in description:
    # Приводим к нижнему регистру и удаляем знаки пунктуации
    i = i.lower()
    i = ''.join([char for char in i if char not in string.punctuation])

    words.extend(word_tokenize(i))

unique_words = list(set(words))
len(unique_words)

24485

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
from nltk.metrics.distance import (
    edit_distance,
    edit_distance_align,
    binary_distance,
    jaccard_distance,
    masi_distance,
    interval_distance,
    custom_distance,
    presence,
    fractional_presence,
)

In [ ]:
import random

random_words = random.sample(unique_words, 10)
for i in range(0, len(random_words)-3, 2):
    distance = edit_distance(random_words[i], random_words[i+1])
    print(f'Расстояние между словами {random_words[i]} и {random_words[i+1]}: {distance}')

Расстояние между словами relyea и tunnel: 5
Расстояние между словами pail и shaily: 3
Расстояние между словами quck и technique: 8
Расстояние между словами wife и kindof: 5


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def nearest_words(word, k):
    dictance_dict = {w:edit_distance(word, w) for w in words}
    sorted_dict = dict(sorted(dictance_dict.items(), key=lambda x: x[1]))
    for key in list(sorted_dict.keys())[:k]:
        print(f'Расстояние от {word} до {key} равно {sorted_dict[key]}')

In [ ]:
words = ['rfeeodmm', 'fereedom', 'fredeom', 'fredom',
         'fredoom', 'frrteddmon']

nearest_words('freedom', 3)

Расстояние от freedom до fereedom равно 1
Расстояние от freedom до fredom равно 1
Расстояние от freedom до fredeom равно 2


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами:
    * word
    * stemmed_word
    * normalized_word

Столбец `word` укажите в качестве индекса.

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# Инициализация SnowballStemmer и WordNetLemmatizer
snb_stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
stemmed_word = [snb_stemmer.stem(word) for word in unique_words]
normalized_word = [lemmatizer.lemmatize(word) for word in unique_words]

# Создание DataFrame с одинаковой длиной данных и индекса
data = {'stemmed_word': stemmed_word, 'normalized_word': normalized_word}
words_df = pd.DataFrame(data, index=unique_words)
words_df.index.name = 'word'

words_df

,stemmed_word,normalized_word
word,,
favourtie,favourti,favourtie
taylor,taylor,taylor
enchilladas,enchillada,enchilladas
flower7,flower7,flower7
originality,origin,originality
...,...,...
maggie,maggi,maggie
minis,mini,mini
slurry,slurri,slurry


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

# Загрузка данных
data = pd.read_csv('preprocessed_descriptions.csv')
description = data.preprocessed_descriptions.fillna('').astype(str)

# Удаление стоп-слов
stop_words = set(stopwords.words('english'))
description_without_stopwords = description.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Подсчет доли стоп-слов в общем количестве слов
total_words = ' '.join(description).split()
stop_words_count = sum(1 for word in total_words if word.lower() in stop_words)
stop_words_fraction = stop_words_count / len(total_words)

print("Доля стоп-слов в общем количестве слов:", stop_words_fraction)

# Топ-10 слов до удаления стоп-слов
all_words = ' '.join(description).split()
word_freq_before_removal = nltk.FreqDist(all_words)
print("Топ-10 слов до удаления стоп-слов:", word_freq_before_removal.most_common(10))

# Топ-10 слов после удаления стоп-слов
all_words_without_stopwords = ' '.join(description_without_stopwords).split()
word_freq_after_removal = nltk.FreqDist(all_words_without_stopwords)
print("Топ-10 слов после удаления стоп-слов:", word_freq_after_removal.most_common(10))

Доля стоп-слов в общем количестве слов: 0.4711034401529359
Топ-10 слов до удаления стоп-слов: [('the', 40413), ('a', 35131), ('and', 30585), ('i', 27945), ('this', 27181), ('to', 23598), ('it', 23300), ('is', 20306), ('of', 18405), ('for', 16023)]
Топ-10 слов после удаления стоп-слов: [('recipe', 15198), ('make', 6438), ('time', 5287), ('use', 4652), ('great', 4522), ('like', 4276), ('easy', 4263), ('one', 4018), ('good', 3887), ('made', 3874)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

random_recipe = data.sample(5)

# Создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование описаний в числовые вектора
tfidf_matrix = vectorizer.fit_transform(random_recipe['preprocessed_descriptions'])

# Получение имен признаков (слов) из TfidfVectorizer
feature_names = vectorizer.get_feature_names_out()

# Создание DataFrame из числовых векторов
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
print(tfidf_df)

        add      also  although   another       any     bacon   barilla  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.387962   
4  0.107773  0.107773  0.107773  0.107773  0.107773  0.107773  0.000000   

         be       big     bread  ...      very      want   website      well  \
0  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
1  0.212982  0.212982  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.238709  ...  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  ...  0.107773  0.107773  0.107773  0.107773   

      which      will      with      work       you       zwt  
0  0

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
import itertools

data = pd.read_csv('preprocessed_descriptions.csv')

# Заполнение пропущенных значений пустой строкой
data['preprocessed_descriptions'] = data['preprocessed_descriptions'].fillna('')


# Создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование описаний в числовые вектора
tfidf_matrix = vectorizer.fit_transform(data['preprocessed_descriptions'])

# Создание всех возможных пар рецептов
pairs = list(itertools.combinations(data['name'], 2))

# Вычисление близости между парами рецептов
similarities = []
for pair in pairs:
    index1 = data.index[data['name'] == pair[0]].tolist()[0]
    index2 = data.index[data['name'] == pair[1]].tolist()[0]
    similarity = 1 - cosine(tfidf_matrix[index1].toarray().ravel(), tfidf_matrix[index2].toarray().ravel())
    similarities.append([pair[0], pair[1], similarity])

# Создание DataFrame для близости между рецептами
similarities_df = pd.DataFrame(similarities, columns=['Recipe 1', 'Recipe 2', 'Similarity'])
print(similarities_df)